In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from thefuzz import fuzz

In [2]:
df_canal = pd.read_csv('./Datasets/CanalDeVenta.csv')
df_clientes = pd.read_csv('./Datasets/Clientes.csv', delimiter=';', encoding='utf-8')
df_compra = pd.read_csv('./Datasets/Compra.csv')
df_gasto = pd.read_csv('./Datasets/Gasto.csv')
df_local = pd.read_csv('./Datasets/Localidades.csv')
df_prov = pd.read_csv('./Datasets/Proveedores.csv', encoding='iso-8859-1')
df_suc = pd.read_csv('./Datasets/Sucursales.csv', delimiter=';')
df_tipo = pd.read_csv('./Datasets/TiposDeGasto.csv')
df_venta = pd.read_csv('./Datasets/Venta.csv')

# Tabla Venta

In [3]:
df_venta.describe()

,IdVenta,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
count,46180.000000,46180.000000,46180.000000,46180.000000,46180.000000,46180.000000,4.526000e+04,45296.000000
mean,23151.534279,2.018038,1502.635232,14.782352,2488.830858,42901.264379,5.876587e+03,2.388246
std,13432.585938,0.745254,972.536404,8.876539,960.231933,83.588487,2.510651e+05,3.562045
min,1.000000,1.000000,1.000000,1.000000,1011.000000,42737.000000,3.000000e+00,1.000000
25%,11545.750000,1.000000,691.000000,7.000000,1574.000000,42833.000000,4.380000e+02,2.000000
50%,23090.500000,2.000000,1359.000000,15.000000,2468.000000,42902.000000,6.540000e+02,2.000000
75%,34635.250000,3.000000,2273.000000,23.000000,3433.000000,42974.000000,1.789000e+03,3.000000
max,47600.000000,3.000000,3407.000000,31.000000,3979.000000,43043.000000,3.373920e+07,300.000000


In [5]:
def get_moda(df, element_id, referencia, target):
    """Obtiene la moda de una columna target filtrando a través de la columna referencia"""
    df_moda = df[df[referencia] == element_id].mode()
    return df_moda[target][0]

In [6]:
def input_moda(df, element_id, referencia, target):
    """Cambia todos los valores de las columnas al valor de la moda"""
    moda = get_moda(df, element_id, referencia, target)
    df.loc[(df[target] != moda) & (df[referencia] == element_id), target] = moda

In [29]:
def fill_moda(df, columna_filtro, target):
    productos = [x for x in df[columna_filtro].unique()]
    for product in productos:
        input_moda(df_venta, product, columna_filtro, target)

In [30]:
fill_moda(df_venta, 'IdProducto', 'Precio')

# Localidad

In [9]:
df_provincias = df_local.loc[:,['provincia_id','provincia_nombre']].drop_duplicates()

# Tabla Clientes

In [10]:
df_clientes = df_clientes.drop(columns='col10')

In [11]:
def get_apellidos(df, columna):
    apellidos = []
    for nombre in df[columna]:
        if isinstance(nombre, str):
            nombre = nombre.upper()
            if ', ' in nombre:
                apellidos.append(nombre.split(', ')[0])
            else:
                words = nombre.split(' ')
                if len(words) > 3:
                    last = words[-2:]
                    apellidos.append(' '.join(last))
                else:
                    apellidos.append(words[-1])
        else:
            apellidos.append(float('NaN'))
    return apellidos

In [12]:
def get_names(element):
    """Obtengo sólo los nombres"""
    if isinstance(element, str):
        element = element.upper()
        if ', ' in element:
            return element.split(', ')[1]

        names_last = element.split(' ')
        if len(names_last) > 3:
            names = names_last[:-2]
        else:
            names = names_last[:-1]

        return ' '.join(names)

In [13]:
def nombre_apellido(df):
    """Creo una columna de apellidos y nombres por separado"""
    apellidos = get_apellidos(df, 'Nombre_y_Apellido')
    df.insert(3, 'Apellido', apellidos)
    df['Nombre_y_Apellido'] = df['Nombre_y_Apellido'].apply(get_names)
    df.rename(columns={'Nombre_y_Apellido':'Nombre'}, inplace=True)


In [14]:
nombre_apellido(df_clientes)

In [15]:
def provincia_to_id(element):
    """Obtengo el Id de la provincia y reemplazo su nombre"""
    if isinstance(element, str):
        mask = df_provincias[df_provincias['provincia_nombre'].apply(lambda x: fuzz.partial_ratio(x,element)) > 80] # Aplico la función de fuzz, que implementa Levinshtein, para obtener el id de la provincia
        return mask['provincia_id'].values.tolist()[0]
    else:
        return element

In [16]:
def idprovincia(df):
    """Aplica la función provincia_to_id a un dataset"""
    df['Provincia'] = df['Provincia'].apply(provincia_to_id)
    df.rename(columns={'Provincia':'IdProvincia'}, inplace=True)
    df['IdProvincia'] = df['IdProvincia'].astype('Int64')

In [17]:
idprovincia(df_clientes)

In [18]:
def replace_comma(stringed):
    """Se reemplazan las comas por puntos, para poder transformar el dato en float"""
    if isinstance(stringed, str) and len(stringed) > 2:
        with_comma = stringed.replace(',','.')
        return float(with_comma)
    else:
        return float('NaN')

In [19]:
def input_replaced_comma(df):
    """Se aplica la función replace_comma a un dataset"""
    df.X = df.X.apply(replace_comma)
    df.Y = df.Y.apply(replace_comma)

In [20]:
input_replaced_comma(df_clientes)

In [21]:
def closer_location(df, cen_lon, cen_lat, lon, lat, target):
    """De acuerdo a las coordenadas del cliente, determino cuál es la localización más cercana"""
    result = np.nan
    for i in range(1,500): # Itero para varios radios
        if len(df[(df[cen_lon].apply(lambda x: int(abs(x-lon)*i)) == 0) & (df[cen_lat].apply(lambda y: int(abs(y-lat)*i)) == 0)]) == 0:
            result = df[(df[cen_lon].apply(lambda x: int(abs(x-lon)*(i-1))) == 0) & (df[cen_lat].apply(lambda y: int(abs(y-lat)*(i-1))) == 0)]
            break
    return result[target].values.tolist()[0]

In [22]:
def get_location(x, df, target_col):
    """Lleno los valores faltantes en localidad o provincia usando las coordenadas"""
    na_local = df[df[target_col].isna()]
    if x.name == target_col:
        names = []
        ind = x.index
        if target_col == 'IdProvincia':
            target = 'provincia_id'
        else:
            target = 'nombre'
        lon = na_local.loc[ind, 'X'].values
        lat = na_local.loc[ind, 'Y'].values
        for i in range(len(lon)):
            names.append(closer_location(df_local, 'centroide_lon', 'centroide_lat', lon[i], lat[i], target))
        return pd.Series(names, ind)
    else:
        return x


In [23]:
def fill_location(df, target):
    df[df[target].isna()] = df[df[target].isna()].apply(lambda x: get_location(x, df, target))

In [24]:
fill_location(df_clientes, 'Localidad')
fill_location(df_clientes, 'IdProvincia')

In [25]:
df_clientes.fillna('None', inplace=True)

# Tabla Compra

In [92]:
def fill_precio(x):
    """Completa los precios faltantes de acuerdo a la media de cada producto"""
    means_product_prices = df_compra.groupby(by='IdProducto').mean().loc[:,'Precio']
    id_prod = df_compra[df_compra['Precio'].isna()]['IdProducto']
    if x.name == 'Precio':
        price = means_product_prices[id_prod]
        return np.round(price.values.tolist(), 2)
    else:
        return x

In [96]:
df_compra[df_compra['Precio'].isna()] = df_compra[df_compra['Precio'].isna()].apply(fill_precio)

# Tabla Sucursales

In [99]:
df_suc

,ID,Sucursal,Direccion,Localidad,Provincia,Latitud,Longitud
0,1,Cabildo,Av. Cabildo 1342,Ciudad de Buenos Aires,Ciudad de Buenos Aires,"-34,5678060","-58,4495720"
1,2,Palermo 1,Guatemala 5701,CABA,CABA,"-34,5790350","-58,4335660"
2,3,Palermo 2,Gral. Lucio Norberto Mansilla 2668,CABA,C deBuenos Aires,"-34,5959660","-58,4051500"
3,4,Corrientes,Av. Corrientes 2352,Ciudad de Buenos Aires,Bs As,"-34,6046850","-58,3987640"
4,5,Almagro,Venezuela 3650,Capital,Bs.As.,"-34,6173080","-58,4161790"
5,6,Caballito,Av. Rivadavia 4708,Capital Federal,Ciudad de Buenos Aires,"-34,6163030","-58,4318490"
6,7,Flores,Av. Rivadavia 5746,CapFed,Buenos Aires,"-34,6228680","-58,4464490"
7,8,Alberdi,Av. Juan Bautista Alberdi 1634,Cap. Fed.,Buenos Aires,"-34,6273060","-58,4514980"
8,9,Deposito,Pedernera 530,Cap. Federal,Buenos Aires,"-34,6351340","-58,4615130"
9,10,Velez,Av. Juan Bautista Justo 7738,Cdad de Buenos Aires,Ciudad de Buenos Aires,"-34,6312570","-58,4980630"
